In [1]:
import pandas as pd

In [142]:
df_cargos_desc = pd.read_excel("cargos, variables original.xlsx", sheet_name=0)
cargos_list = list(df_cargos_desc["CÓDIGO"])
#cargos_list

In [2]:
data = pd.read_csv("total_data_clean.csv", index_col=0)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250 entries, 0 to 249
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id_resol                     250 non-null    object 
 1   año                          250 non-null    float64
 2   cargos                       250 non-null    object 
 3   códigos                      245 non-null    object 
 4   Año                          250 non-null    int64  
 5   Categoría                    250 non-null    object 
 6   Materia                      250 non-null    object 
 7   Empresa denunciada           250 non-null    object 
 8   Criterios aplicables         250 non-null    object 
 9   Medios probatoriosCargo      250 non-null    object 
 10  Medios probatorios Descargo  250 non-null    object 
 11  Cuestiones                   250 non-null    object 
 12  Normativa                    250 non-null    object 
 13  Multas              

In [3]:
# crear lista de códigos de cargos multicategoricos
data["códigos"] = data["códigos"].apply(lambda x: x.split(",") if isinstance(x, str) else [])

In [4]:
data.head()

,id_resol,año,cargos,códigos,Año,Categoría,Materia,Empresa denunciada,Criterios aplicables,Medios probatoriosCargo,Medios probatorios Descargo,Cuestiones,Normativa,Multas,Medidas correctivas,Configuración del Tribunal,Decisión,dec_fav_denunciante,cargos limpio
0,0002-2020/SPC-INDECOPI,2020.0,Luego de contratar los servicios de la denunci...,[ser1],2020,Pérdida de equipaje,Idoneidad del servicio,EMPRESA DE TRANSPORTES TURISMO SEÑOR DE ATACO ...,"""De esta manera, al no haber quedado acreditad...",0,0,0,Ley No. 27444 Ley del Procedimiento Administra...,0,0,Javier Eduardo Raymundo Villa García Vargas Ju...,1,0,luego contratar servicios denunciada 26 octubr...
1,0944-2020/SPC-INDECOPI,2020.0,La empresa no habría brindado un servicio idón...,[ser3],2020,Pérdida de equipaje,Idoneidad del servicio,EMPRESA DE TRANSPORTES Y SERVICIOS MÚLTIPLES S...,"""Dicho esto, este Colegiado considera que el f...",0,0,0,Código de Protección y Defensa del Consumidor ...,50 UIT,0,Javier Eduardo Raymundo Villa García Vargas Ro...,1,1,empresa brindado servicio idóneo ruta trujillo...
2,1127-2020/SPC-INDECOPI,2020.0,el bus partió con treinta (30) minutos de retr...,"[ser1, ser13]",2020,Pérdida de equipaje,Idoneidad del servicio,SERVICIO Y TURISMO EXPRES VÍA NACIONAL S.A.C.,"""En tal sentido, la denunciada, en principio, ...",0,0,0,Código de Protección y Defensa del Consumidor ...,"1,5 UIT",0,Javier Eduardo Raymundo Villa García Vargas Ju...,1,1,bus partió treinta 30 minutos retraso viaje bu...
3,1459-2019/SPC-INDECOPI,2019.0,(i) La denunciada no cumplía con redactar con ...,"[o5, c3, c8]",2019,Pérdida de equipaje,Idoneidad del servicio,EMPRESA DE TRANSPORTES PERÚ BUS S.A.,"""En consecuencia, corresponde revocar la resol...",0,0,0,Código de Protección y Defensa del Consumidor ...,0,0,Javier Eduardo Raymundo Villa García Vargas Ju...,0,0,i denunciada cumplía redactar claridad sencill...
4,1460-2019/SPC-INDECOPI,2019.0,(i) La denunciada no cumplía con redactar con ...,"[o5, c3, c8]",2019,Pérdida de equipaje,Idoneidad del servicio,TRANSPORTES SOYUZ S.A.,"""En consecuencia, corresponde revocar la resol...",0,0,0,Código de Protección y Defensa del Consumidor ...,0,0,Javier Eduardo Raymundo Villa García Vargas Ju...,0,0,i denunciada cumplía redactar claridad sencill...


# Separación del conjunto de datos en entrenamiento y test

In [15]:
from sklearn.model_selection import train_test_split

# Separación del conjunto de datos en entrenamiento y test
# Separar el 20% para el test

X_train, X_test, Y_train, Y_test = train_test_split(data.drop("dec_fav_denunciante", 
                                                            axis=1),
                                                    data["dec_fav_denunciante"],
                                                    test_size=0.2,
                                                    random_state=10)

print("training set size: ", X_train.shape[0])
print("test set size: ", X_test.shape[0])

training set size:  200
test set size:  50


In [78]:
from sklearn.pipeline import Pipeline

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer


from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [31]:
# Crear un generador de cross-validation: 5-fold
kf = KFold(n_splits=5, random_state=0, shuffle=True);

# Optimización de hiperparámetros mediante CV

In [54]:
# crear DF para almacenar todos los resultados de los experiemntos
experiments_results = pd.DataFrame(
    columns=['params', 'mean_test_accuracy', 'std_test_accuracy', 'model_name'])

In [75]:
# función que automatiza el proceso de optimización de hiperparámetros meidante CV para un
# modelo, retorna un DF con los resultados de los experimentos

def run_CV(model, model_name, parameters, cv_iter, X, Y):
    
    clf = GridSearchCV(estimator = model,       # especificar el modelo a entrenar
                       param_grid = parameters, # especificar los parámetros a optimizar
                       cv = cv_iter,            # 5-fold cross validation
                       scoring = ['accuracy'],  # definir métricas de evaluación
                       refit = False            # no entrenar un modelo con todos los datos
                      )

    # run the cv experiments
    clf.fit(X, Y)

    # store the results of the experiments in a dataframe
    keys = ['mean_test_accuracy', 'std_test_accuracy', 'params']
    results_dic = {k:v for k, v in clf.cv_results_.items() if k in keys}
    results = pd.DataFrame(results_dic)
    results["model_name"] = model_name
    
    return results

### Experimentos modelo RF + n-gramas

In [76]:
# crear modelo combinado de n-gramas y random forest
column_trans = ColumnTransformer([
    # Applicar una transformación CountVectorizer a la columna 'cargos'  
    ('CargosLimpioNGramas',  CountVectorizer(), 'cargos limpio')
    ],
    remainder='drop' # ignore the remaining columns
)

pipeline = Pipeline([
    # Primer paso: extracción de características
    ("FeatureExtraction", column_trans),
    # Segundo paso: predicción de la decisión
    ('RandForestPred', RandomForestClassifier(random_state=0)) # freeze random state
])

In [73]:
# especificación de hiperparámetros a optimizar
parameters = {
    'FeatureExtraction__CargosLimpioNGramas__max_features': [5, 25, 50, None], # cant de features a extraer
    'FeatureExtraction__CargosLimpioNGramas__max_df': [1.0, 0.9], 
    'FeatureExtraction__CargosLimpioNGramas__min_df': [0.0, 0.05],
    'FeatureExtraction__CargosLimpioNGramas__ngram_range': [(1, 1), (2, 2), (3, 3)], # n-grams length
    'RandForestPred__criterion': ['gini', 'entropy'], 
    'RandForestPred__min_samples_leaf':[1, 3, 5, 7],
    'RandForestPred__bootstrap': [True, False]
}

In [74]:
exp_results_df = run_CV(pipeline, 
                        "N-gramas + RF", 
                        parameters, 
                        kf, 
                        X_train, Y_train)

experiments_results = experiments_results.append(exp_results_df)

,params,mean_test_accuracy,std_test_accuracy,model_name
0,"{'RandForestPred__bootstrap': True, 'RandFores...",0.675,0.082158,N-gramas + RF
1,"{'RandForestPred__bootstrap': True, 'RandFores...",0.675,0.061237,N-gramas + RF
2,"{'RandForestPred__bootstrap': False, 'RandFore...",0.670,0.050990,N-gramas + RF
3,"{'RandForestPred__bootstrap': False, 'RandFore...",0.670,0.085732,N-gramas + RF


### Experimentos modelo SVC + n-gramas

In [ ]:
# crear modelo combinado de n-gramas y random forest
column_trans = ColumnTransformer([
    # Applicar una transformación CountVectorizer a la columna 'cargos'  
    ('CargosLimpioNGramas',  CountVectorizer(), 'cargos limpio')
    ],
    remainder='drop' # ignore the remaining columns
)

pipeline = Pipeline([
    # Primer paso: extracción de características
    ("FeatureExtraction", column_trans),
    # Segundo paso: predicción de la decisión
    ('SVMPred', SVC(random_state=0)) # freeze random state
])

In [ ]:
# especificación de hiperparámetros a optimizar
parameters = {
    'FeatureExtraction__CargosLimpioNGramas__max_features': [5, 25, 50, None], # cant de features a extraer
    'FeatureExtraction__CargosLimpioNGramas__max_df': [1.0, 0.9], 
    'FeatureExtraction__CargosLimpioNGramas__min_df': [0.0, 0.05],
    'FeatureExtraction__CargosLimpioNGramas__ngram_range': [(1, 1), (2, 2), (3, 3)], # n-grams length
    
    'SVMPred__C': [0.01, 0.1, 1, 10, 100], 
    'SVMPred__kernel':['linear', 'rbf'],
}

In [ ]:
exp_results_df = run_CV(pipeline, 
                        "N-gramas + SVC", 
                        parameters, 
                        kf, 
                        X_train, Y_train)

experiments_results = experiments_results.append(exp_results_df)

### Experimentos modelo RF + TF-IDF

In [ ]:
# crear modelo combinado de n-gramas (TF-IDF) y random forest
column_trans_2 = ColumnTransformer([
    # Applicar una transformación TFIDF a la columna 'cargos'  
    ('CargosLimpiosTFIDF',  TfidfVectorizer(), 'cargos limpio')
    ],
    remainder='drop' # ignore the remaining columns
)

pipeline_2 = Pipeline([
    # Primer paso: extracción de características
    ("FeatureExtraction", column_trans_2),
    # Segundo paso: predicción de la decisión
    ('RandForestPred', RandomForestClassifier(random_state=0)) # freeze random state
])

In [ ]:
# especificación de hiperparámetros a optimizar
parameters_2 = {
    # parámetros del la transformación de la columna Cargos Limpio
    'FeatureExtraction__CargosLimpiosTFIDF__ngram_range': [(1, 1), (2, 2), (3, 3)], # n-grams length
    'FeatureExtraction__CargosLimpiosTFIDF__max_features': [5, 25, 50, None], # cant de features a extraer
    'FeatureExtraction__CargosLimpiosTFIDF__max_df': [1.0, 0.9], 
    'FeatureExtraction__CargosLimpiosTFIDF__min_df': [0.0, 0.05],

    'RandForestPred__criterion': ['gini', 'entropy'], 
    'RandForestPred__min_samples_leaf':[1, 3, 5, 7],
    'RandForestPred__bootstrap': [True, False]
}

In [ ]:
exp_results_df = run_CV(pipeline_2, 
                        "TF-IDF + RF", 
                        parameters_2, 
                        kf, 
                        X_train, Y_train)

experiments_results = experiments_results.append(exp_results_df)

### Experimentos modelo SVC + TF-IDF

In [ ]:
# crear modelo combinado de n-gramas (TF-IDF) y SVC
column_trans = ColumnTransformer([
    # Applicar una transformación TFIDF a la columna 'cargos'  
    ('CargosLimpiosTFIDF',  TfidfVectorizer(), 'cargos limpio')
    ],
    remainder='drop' # ignore the remaining columns
)

pipeline = Pipeline([
    # Primer paso: extracción de características
    ("FeatureExtraction", column_trans),
    # Segundo paso: predicción de la decisión
    ('SVMPred', SVC(random_state=0)) # freeze random state
])

In [ ]:
# especificación de hiperparámetros a optimizar
parameters = {
    # parámetros del la transformación de la columna Cargos Limpio
    'FeatureExtraction__CargosLimpiosTFIDF__ngram_range': [(1, 1), (2, 2), (3, 3)], # n-grams length
    'FeatureExtraction__CargosLimpiosTFIDF__max_features': [5, 25, 50, None], # cant de features a extraer
    'FeatureExtraction__CargosLimpiosTFIDF__max_df': [1.0, 0.9], 
    'FeatureExtraction__CargosLimpiosTFIDF__min_df': [0.0, 0.05],

    'SVMPred__C': [0.01, 0.1, 1, 10, 100], 
    'SVMPred__kernel':['linear', 'rbf'],
}

In [ ]:
exp_results_df = run_CV(pipeline, 
                        "TF-IDF + SVC", 
                        parameters, 
                        kf, 
                        X_train, Y_train)

experiments_results = experiments_results.append(exp_results_df)

### Experimento Cargos Multicat + RF

In [126]:
# entrenar el multi label binarizer
mlb = MultiLabelBinarizer(classes=cargos_list)
mlb.fit([cargos_list])

# hay algunos errores que se debe corregir en la codificación
X_train_proc = mlb.transform(X_train["códigos"])

print(X_train_proc.shape)

(200, 40)


c:\users\diego\appdata\local\programs\python\python38\lib\site-packages\sklearn\preprocessing\_label.py:994: UserWarning: unknown class(es) ['', 'cc3', 'o6', 'otros', 'se11', 'seg5\n', 'ser2ser7'] will be ignored
  warnings.warn('unknown class(es) {0} will be ignored'


In [133]:
# crear modelo combinado de cargos (multicat) y random forest

pipeline_3 = Pipeline([
    # predicción de la decisión
    ('RandForestPred', RandomForestClassifier(random_state=0)) # freeze random state
])

In [134]:
# especificación de hiperparámetros a optimizar
parameters_3 = {
    'RandForestPred__criterion': ['gini', 'entropy'], 
    'RandForestPred__min_samples_leaf':[1, 3, 5, 7],
    'RandForestPred__bootstrap': [True, False]
}

In [135]:
exp_results_df = run_CV(pipeline_3, 
                        "CargosMulticat + RF", 
                        parameters_3, 
                        kf, 
                        X_train_proc, Y_train)

experiments_results = experiments_results.append(exp_results_df)

### Experimento Cargos Multicat + RF

In [137]:
# crear modelo combinado de cargos (multicat) y SVM
pipeline = Pipeline([
    # predicción de la decisión
    ('SVMPred', SVC(random_state=0)) # freeze random state
])

In [139]:
# especificación de hiperparámetros a optimizar
parameters = {
    'SVMPred__C': [0.01, 0.1, 1, 10, 100], 
    'SVMPred__kernel':['linear', 'rbf'],
}

In [140]:
exp_results_df = run_CV(pipeline, 
                        "CargosMulticat + RF", 
                        parameters, 
                        kf, 
                        X_train_proc, Y_train)

experiments_results = experiments_results.append(exp_results_df)

In [ ]:
experiments_results.to_csv("resultados_experimentos.csv")